# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [23]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [24]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,keetmanshoop,-26.5833,18.1333,14.51,47,40,6.17,NaN,1717687822
1,1,archangelos,36.2149,28.1149,31.99,24,82,5.38,GR,1717687823
2,2,rantauprapat,2.1000,99.8333,23.50,96,75,1.18,ID,1717687823
3,3,senatobia,34.6176,-89.9687,27.36,86,7,3.27,US,1717687823
4,4,preobrazheniye,42.9019,133.9064,11.58,98,41,0.57,RU,1717687824


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [30]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng','Lat',
    geo = True,
    tiles = 'OSM',
    color='City',
    size = 'Humidity',
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    alpha = 0.7,
    hover_cols = ['City','Humidity']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [26]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[(city_data_df['Max Temp'] < 30) &
                                  (city_data_df['Humidity'] >= 40) &
                                  (city_data_df['Humidity'] <= 60) &
                                  (city_data_df['Cloudiness'] >= 40) &
                                  (city_data_df['Cloudiness'] <= 60) &
                                  (city_data_df['Wind Speed'] < 9)]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,stefan voda,46.5153,29.6631,23.29,56,59,2.96,MD,1717687827
16,16,stanley,54.8680,-1.6985,15.10,54,40,6.69,GB,1717687679
62,62,hinton,53.4001,-117.5857,12.84,42,51,2.41,CA,1717687845
171,171,andergrove,-21.0833,149.1833,14.16,55,46,3.11,AU,1717687884
217,217,kara suu,40.7046,72.8667,22.98,46,40,6.00,KG,1717687901
330,330,tafresh,34.6920,50.0130,19.17,46,53,2.89,IR,1717687940
366,366,aalborg,57.0480,9.9187,13.96,58,40,7.72,DK,1717687951
454,454,chipata,-13.6333,32.6500,21.22,40,49,2.74,ZM,1717687981
456,456,lamar,33.6668,-95.5836,28.83,59,40,0.00,US,1717687981
521,521,coahuayana de hidalgo,18.7000,-103.6583,29.84,51,47,1.72,MX,1717688004


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(filtered_cities_df)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,15,stefan voda,46.5153,29.6631,23.29,56,59,2.96,MD,1717687827,
16,16,stanley,54.8680,-1.6985,15.10,54,40,6.69,GB,1717687679,
62,62,hinton,53.4001,-117.5857,12.84,42,51,2.41,CA,1717687845,
171,171,andergrove,-21.0833,149.1833,14.16,55,46,3.11,AU,1717687884,
217,217,kara suu,40.7046,72.8667,22.98,46,40,6.00,KG,1717687901,
330,330,tafresh,34.6920,50.0130,19.17,46,53,2.89,IR,1717687940,
366,366,aalborg,57.0480,9.9187,13.96,58,40,7.72,DK,1717687951,
454,454,chipata,-13.6333,32.6500,21.22,40,49,2.74,ZM,1717687981,
456,456,lamar,33.6668,-95.5836,28.83,59,40,0.00,US,1717687981,
521,521,coahuayana de hidalgo,18.7000,-103.6583,29.84,51,47,1.72,MX,1717688004,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [28]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories" : "accommodation.hotel",
          "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}",
    params["bias"] =  f"proximity:{lng},{lat}"   

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()

    print(f"Resoibse for {row['City']}:", name_address)

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
    # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Resoibse for stefan voda: {'type': 'FeatureCollection', 'features': []}
stefan voda - nearest hotel: No hotel found
Resoibse for stanley: {'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'Hotel 52', 'country': 'United Kingdom', 'country_code': 'gb', 'state': 'England', 'state_district': 'North East', 'county': 'County Durham', 'city': 'Stanley', 'village': 'Tanfield Lea', 'hamlet': 'West Shield Row Villas', 'postcode': 'DH9 0PG', 'street': 'Cemetery Road', 'lon': -1.694563255467638, 'lat': 54.87283945, 'state_code': 'ENG', 'formatted': 'Hotel 52, Cemetery Road, Stanley, DH9 0PG, United Kingdom', 'address_line1': 'Hotel 52', 'address_line2': 'Cemetery Road, Stanley, DH9 0PG, United Kingdom', 'categories': ['accommodation', 'accommodation.hotel', 'building', 'building.accommodation'], 'details': ['details'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Lice

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
15,15,stefan voda,46.5153,29.6631,23.29,56,59,2.96,MD,1717687827,No hotel found
16,16,stanley,54.8680,-1.6985,15.10,54,40,6.69,GB,1717687679,Hotel 52
62,62,hinton,53.4001,-117.5857,12.84,42,51,2.41,CA,1717687845,Timberland Hotel
171,171,andergrove,-21.0833,149.1833,14.16,55,46,3.11,AU,1717687884,No hotel found
217,217,kara suu,40.7046,72.8667,22.98,46,40,6.00,KG,1717687901,No hotel found
330,330,tafresh,34.6920,50.0130,19.17,46,53,2.89,IR,1717687940,هتل پردیسان
366,366,aalborg,57.0480,9.9187,13.96,58,40,7.72,DK,1717687951,Hotel Jomfru Ane
454,454,chipata,-13.6333,32.6500,21.22,40,49,2.74,ZM,1717687981,Kamocho Gueshouse
456,456,lamar,33.6668,-95.5836,28.83,59,40,0.00,US,1717687981,1st St Lofts
521,521,coahuayana de hidalgo,18.7000,-103.6583,29.84,51,47,1.72,MX,1717688004,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [29]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng','Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    scale = 2,
    color='City',
    size = 'Humidity',
    alpha = 0.7,
    hover_cols = ['City','Humidity','Hotel Name', 'Country']
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)